___
[Home- go back to hadardestani.com](http://hadiardestani.com/)
___
# Data Cleaning and Preparation

A few years ago I borrowed some money from Lending club while I was in school and needed some money. I was very happy with the process and how fast everything worked out. And of course, I paid it off to the last penny. But this is not always the case. Some people default on their loans. How does lending club trust you to borrow money? Of course, they use their past Data, consumer Data, and statistics to determine what is the probability of a potential borrower to pay back in full, and based on their formula they approve or reject the request. I will try to download publically available data from their website and try to create a model that will help predict whether if a potential borrower will pay back in full or not. 

I will be using their data from 2019 Q4 (I could download the whole year or even a few years of data but it would be really hard on my machine running big datasets. This smaller dataset will serve the purpose here!) and try to classify and predict whether or not the borrower paid back their loan in full. I downloaded the data from [here](https://www.lendingclub.com/info/download-data.action) (of course I had a username and password since I once was a browser so I just had to reset my password.) At the end of this practice I will download the end result dataset so I can use it for modeling an algorithm in another post.

# Libraries

***Let's first import the usual libraries***

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

## The Data

***I am going to use pandas to import my CSV file***

In [18]:
lending_club = pd.read_csv('/Users/hadiardestani/Desktop/lending new/LoanStats_2019Q4.csv',skiprows=1,low_memory=False)

In [19]:
#I am going to work off of a copy of the data frame, so my machine doesn't have to re-read the entire data frame every time.
#this will save me some memory.
df = lending_club.copy(deep=True)
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,164027473,NaN,20000.0,20000.0,20000.0,36 months,12.40%,668.12,B,B4,...,393.88,19282.07,668.12,N,NaN,NaN,NaN,NaN,NaN,NaN
1,163984413,NaN,16500.0,16500.0,16500.0,60 months,10.33%,353.27,B,B1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,164193225,NaN,7500.0,7500.0,7500.0,36 months,12.40%,250.55,B,B4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,162948736,NaN,19000.0,19000.0,18975.0,36 months,6.46%,581.99,A,A1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,164141013,NaN,9800.0,9800.0,9800.0,36 months,23.05%,379.62,D,D3,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


***let's check the data***

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128260 entries, 0 to 128259
Columns: 150 entries, id to settlement_term
dtypes: float64(112), object(38)
memory usage: 146.8+ MB


In [21]:
df.shape

(128260, 150)

**There are 150 columns. we need to look into this and see if there are columns that we can eliminate to make the job easier and faster. To achieve this, we will do a series of dataset evaluation tasks. I also downloaded the data dictionary that is available for download as well. In order to get a better grasp of what these columns are we will need to look at it.**

In [22]:
dic_des = pd.read_excel('/Users/hadiardestani/Desktop/lending new/LCDataDictionary.xlsx').dropna()

**Now I am going to add a description column from my imported data dictionary by creating a data frame that only shows names and the description. Imaging doing a copy and transpose paste of all the columns in the dataset and then doing a VLOOKUP of name to bring in the description next to it.**

In [23]:
dic_des = dic_des.rename(columns={'LoanStatNew': 'Name'})
df_dtypes = pd.DataFrame(df.dtypes,columns=['dtypes'])
df_dtypes = df_dtypes.reset_index()
df_dtypes['Name'] = df_dtypes['index']
df_dtypes = df_dtypes[['Name','dtypes']]
vlookup = df_dtypes.merge(dic_des, on='Name',how='left')

In [24]:
vlookup.style.set_properties(subset=['Description'], **{'width': '1000px'})

,Name,dtypes,Description
0,id,object,A unique LC assigned ID for the loan listing.
1,member_id,float64,A unique LC assigned Id for the borrower member.
2,loan_amnt,float64,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."
3,funded_amnt,float64,The total amount committed to that loan at that point in time.
4,funded_amnt_inv,float64,The total amount committed by investors for that loan at that point in time.
5,term,object,The number of payments on the loan. Values are in months and can be either 36 or 60.
6,int_rate,object,Interest Rate on the loan
7,installment,float64,The monthly payment owed by the borrower if the loan originates.
8,grade,object,LC assigned loan grade
9,sub_grade,object,LC assigned loan subgrade


**Let's start by looking at the columns and their description to see if there are columns that won't contribute to our purpose so we acn remove them. This is time consuming but worth it. After going through them I decided below colomuns can be removed. They were chosen because either they had nothing to do with a brrower paying back the ooan such as Id or member id, or anything that is about feature meaning after the loan is approved, or anything that is bad formatted and is redundant.
**id / member_id / funded_amnt / funded_amnt_inv / sub_grade / int_rate./ emp_title / issued_d / zip_code / out_prncp  / out_prncp_inv./ total_pymnt / total_pymnt_inv/ total_rec_prncp / total_rec_int/ total_rec_late_fee / recoveries/ collection_recovery_fee/ last_pymnt_d/ last_pymnt_amnt**

In [25]:
df=df.drop (['id','int_rate','total_pymnt','total_rec_late_fee','member_id','sub_grade','total_pymnt_inv',
             'recoveries','funded_amnt','emp_title','total_rec_prncp','collection_recovery_fee','funded_amnt_inv',
             'issue_d','total_rec_int','last_pymnt_d','zip_code','out_prncp','last_pymnt_amnt','out_prncp_inv',
             'installment','last_credit_pull_d','last_fico_range_high','last_fico_range_low','sec_app_fico_range_low','sec_app_fico_range_high','loan_amnt',
             'next_pymnt_d','num_tl_120dpd_2m','num_tl_30dpd','url'],axis=1)

**Also, there are a few columns that I like to look dipper into them. FICO scores columns. I realized that there are 6 columns for FICO score. fico_range_low, fico_range_high, last_fico_range_low,last_fico_range_high,sec_app_fico_range_low	,sec_app_fico_range_high. It seems that when you apply for a loan Lending club assigns you with a FICO score limit, meaning a lower and higher range. As all the other institutions that borrow money, Lending club updates your score once in a while and marks it by last_. We for sure don't need the 2 columns that gets updated after the loan is approved since they are in the future and don't have anything to do with deciding to approve the loan or not, the same goes for sec_app_fico_range ones. I would like to combine the other 2 columns into one FICO score. Maybe the best way to do it is by getting an average of the 2 number. Let's take a look. **

In [26]:
df['FICO']=(df['fico_range_high']+df['fico_range_low']) /2

In [27]:
fico_columns = ['fico_range_low','fico_range_high','FICO']
df[fico_columns].head()

,fico_range_low,fico_range_high,FICO
0,665.0,669.0,667.0
1,775.0,779.0,777.0
2,710.0,714.0,712.0
3,785.0,789.0,787.0
4,665.0,669.0,667.0


In [28]:
df=df.drop (['fico_range_low','fico_range_high'],axis=1)

**let’s first look at the summary of the. Below function was written by Leonardo Ferreira (Lending Club - Insightful Financial EDA), and it perfectly does the summarizing job. There are many other ways of doing that, but I found this function easy and quick.**

In [29]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary

In [30]:
resumetable(df)

Dataset Shape: (128260, 118)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value,Entropy
0,term,object,2,2,36 months,60 months,36 months,0.92
1,grade,object,2,4,B,B,B,1.96
2,emp_length,object,11233,11,< 1 year,NaN,7 years,3.05
3,home_ownership,object,2,4,RENT,MORTGAGE,RENT,1.41
4,annual_inc,float64,2,10082,13000,26000,159000,8.04
5,verification_status,object,2,3,Verified,Source Verified,Not Verified,1.31
6,loan_status,object,2,7,Current,Current,Fully Paid,0.45
7,pymnt_plan,object,2,1,n,n,n,0.00
8,desc,float64,128260,0,NaN,NaN,NaN,0.00
9,purpose,object,2,12,small_business,debt_consolidation,credit_card,1.91


**Now that we have 118 columns or features. let’s start by handling missing values and first removing the columns that have too many null values that practically we won't be able to use them. As it can be seen in the summary query result from row 100 to 129 the total number of missing values are way more than the total number of data in one column. So, we might be able to remove some of these columns. Let’s dig in.**

In [32]:
#Let's see how many columns contain null values and how many don't.
print("columns with missing values:", len(df.columns[df.isna().any()]))
print("columns with no missing values:", len(df.columns[df.notna().all()]))
print("All columns in the dataframe:",len(df.columns))

columns with missing values: 118
columns with no missing values: 0
All columns in the dataframe: 118


**Now let’s calculate the Percentage of missing values in each column. I am going to modify my summary function and try to use is to calculate this**

In [33]:
null_v = df.isnull().sum()
null_v_per = 100 * df.isnull().sum() / len(df)
#now let's create table view of the the two side by side
missing_value_table = pd.DataFrame({'Count': null_v, 'Percent': null_v_per })
missing_value_table[missing_value_table['Count']>2].sort_values('Percent',ascending=False).head(50)

,Count,Percent
desc,128260,100.000000
settlement_term,128258,99.998441
debt_settlement_flag_date,128258,99.998441
settlement_percentage,128258,99.998441
settlement_amount,128258,99.998441
settlement_date,128258,99.998441
settlement_status,128258,99.998441
sec_app_mths_since_last_major_derog,123235,96.082177
mths_since_last_record,115316,89.907999
hardship_loan_status,114612,89.359114


**As it can be seen there are column or features withn more than 80% missing or null value in them. These feature won't be helping us. So I will drrop any column that has a null value percentage of above %80**

In [34]:
#too_many_null = [i for i in df.count()<128260 *0.80]
#df.drop(df.columns[too_many_null],axis=1,inplace=True)

df = df.dropna(axis=1, thresh=int(0.80*len(df)))

In [35]:
print("columns with missing values:", len(df.columns[df.isna().any()]))
print("columns with no missing values:", len(df.columns[df.notna().all()]))
print("All columns in the dataframe:",len(df.columns))

columns with missing values: 78
columns with no missing values: 0
All columns in the dataframe: 78


In [36]:
#let's check the summary again
resumetable(df)

Dataset Shape: (128260, 78)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value,Entropy
0,term,object,2,2,36 months,60 months,36 months,0.92
1,grade,object,2,4,B,B,B,1.96
2,emp_length,object,11233,11,< 1 year,NaN,7 years,3.05
3,home_ownership,object,2,4,RENT,MORTGAGE,RENT,1.41
4,annual_inc,float64,2,10082,13000,26000,159000,8.04
5,verification_status,object,2,3,Verified,Source Verified,Not Verified,1.31
6,loan_status,object,2,7,Current,Current,Fully Paid,0.45
7,pymnt_plan,object,2,1,n,n,n,0.00
8,purpose,object,2,12,small_business,debt_consolidation,credit_card,1.91
9,title,object,2,12,Business,Debt consolidation,Credit card refinancing,1.91


**From above we can see that there are a few columns that have only one unique value. Those also won't be helpful in our modeling. We will dropm them as well.**

In [37]:
df = df.loc[:,df.apply(pd.Series.nunique) != 1]

There are also a few columns with more than 1 unique value but one of the unique values has been repeated many times. so, the other ones don’t matter much. That makes these columns similar to the ones with only one unique value. So, we can drop them too. I checked those for columns that had less than 6 unique value and end up with going with the ones with only 4.

In [38]:
for i in df.columns:
    if (len(df[i].unique())<4):
        print(df[i].value_counts())
        print()

 36 months    85338
 60 months    42920
Name: term, dtype: int64

w    121116
f      7142
Name: initial_list_status, dtype: int64

Individual    112118
Joint App      16140
Name: application_type, dtype: int64

0.0    128236
1.0        22
Name: acc_now_delinq, dtype: int64

N    111953
Y     13539
Name: hardship_flag, dtype: int64

N    128256
Y         2
Name: debt_settlement_flag, dtype: int64



In [39]:
df=df.drop (['debt_settlement_flag','acc_now_delinq',
'initial_list_status'],axis=1)

In [51]:
df.shape

(128260, 72)

**Lets see how many more columns that have a missding values are left and what is their data type. 

In [56]:
null_v2 = df.isnull().sum()
null_v2_Data_type=null_v2.dtypes
null_v_per2 = 100 * df.isnull().sum() / len(df)
#now let's create table view of the the two side by side
missing_value_table2 = pd.DataFrame({'Count': null_v2, 'Percent': null_v_per2, 'Data_type':null_v2_Data_type })
final_missning=missing_value_table2[missing_value_table2['Count']>2].sort_values('Percent',ascending=False).head(50)
final_missning

,Count,Percent,Data_type
il_util,17065,13.305005,int64
mths_since_recent_inq,14384,11.214720,int64
emp_length,11233,8.757992,int64
mths_since_rcnt_il,3312,2.582255,int64
mo_sin_old_il_acct,3312,2.582255,int64
hardship_flag,2768,2.158116,int64
bc_util,1632,1.272415,int64
percent_bc_gt_75,1564,1.219398,int64
bc_open_to_buy,1562,1.217839,int64
mths_since_recent_bc,1456,1.135194,int64


**All 14 above columns with missing values have a data type of integer. We can easily find a solution to handel the missing values in this columns. 

In [62]:
final_missning2 = final_missning.reset_index()
final_missning2['Name'] = final_missning2['index']
final_missning2 = final_missning2[['Name','Data_type']]
vlookup2 = final_missning2.merge(dic_des2, on='Name',how='left')
vlookup2.style.set_properties(subset=['Description'], **{'width': '1000px'})

,Name,Data_type,Description
0,il_util,int64,Ratio of total current balance to high credit/credit limit on all install acct
1,mths_since_recent_inq,int64,Months since most recent inquiry.
2,emp_length,int64,Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
3,mths_since_rcnt_il,int64,Months since most recent installment accounts opened
4,mo_sin_old_il_acct,int64,Months since oldest bank installment account opened
5,hardship_flag,int64,Flags whether or not the borrower is on a hardship plan
6,bc_util,int64,Ratio of total current balance to high credit/credit limit for all bankcard accounts.
7,percent_bc_gt_75,int64,Percentage of all bankcard accounts > 75% of limit.
8,bc_open_to_buy,int64,Total open to buy on revolving bankcards.
9,mths_since_recent_bc,int64,Months since most recent bankcard account opened.
